In [1]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.curdir, '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
import numpy as np
from src.ik_net import *
from pydrake.all import (
    MultibodyPlant,
    StartMeshcat,
)
from src.utils import RepoDir, BuildEnv, extract_xyzrpy


In [ ]:
meshcat = StartMeshcat()
diagram = BuildEnv(meshcat, directives_file = os.path.join(RepoDir(), "models/iiwa_collision.yaml"))
ee_frame = diagram.GetSubsystemByName("plant").GetFrameByName("body")

dataset = IKDataset(diagram, ee_frame)

joint_angles, targets = dataset.create_data(5**7, manifold = [1, 1, 1], matrix=True)

X = np.concatenate([targets, joint_angles[:, 0:1]], axis=1)
y = joint_angles

model = LinearModule(X.shape[1], 7, hidden_layers = [64, 64], dropout_rate=0.0)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
loss_fn = torch.nn.MSELoss()



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


losses = train_model(model, optimizer, loss_fn, X_train, y_train, epochs=1000, batch_size=2048)
evaluate_model(model, loss_fn, X_test, y_test, dataset)

INFO:drake:Meshcat listening for connections at http://localhost:7002
100%|██████████| 78125/78125 [00:00<00:00, 114083.74it/s]


Training with batching (batch_size=2048)


  0%|          | 2/1000 [00:00<03:26,  4.83it/s]

Epoch 0, Loss: 1.912954


 10%|█         | 102/1000 [00:18<02:37,  5.70it/s]

...

Epoch 800, Loss: 0.854303


 90%|█████████ | 902/1000 [02:49<00:17,  5.60it/s]

Epoch 900, Loss: 0.851382


100%|██████████| 1000/1000 [03:07<00:00,  5.33it/s]


ValueError: operands could not be broadcast together with shapes (6,) (12,) 

In [10]:
print(X_test.shape, y_test.shape)
evaluate_model(model, loss_fn, X_test, y_test, dataset, matrix=True)


(15625, 13) (15625, 7)
Backwards Loss: 1.837712
Position Loss: 0.205746
Test Loss: 0.905109


In [3]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
# plt.yscale('log')  # optional: useful if loss decreases a lot
plt.show()

NameError: name 'losses' is not defined

In [ ]:
X_tensor = torch.FloatTensor(X)
y_tensor = torch.FloatTensor(y)

optimizer.zero_grad()
print(y_tensor[1])
model(X_tensor[1])

tensor([ 2.6931,  1.6616, -1.9657,  0.9957, -2.2489,  0.6748,  2.9113])


tensor([ 2.6505,  2.0579, -1.6534,  0.7292,  0.1248,  0.7395, -0.0949],
       grad_fn=<ViewBackward0>)

In [ ]:
joint_angles[1]

array([ 2.69314317,  1.66155257, -1.96566807,  0.99573811, -2.24894115,
        0.67480338,  2.91133211])

In [ ]:
def closure():
    optimizer.zero_grad()
    pred = model(torch.FloatTensor(X_train))
    loss = loss_fn(pred, torch.FloatTensor(y_train))
    loss.backward()
    return loss

optimizer = torch.optim.LBFGS(model.parameters(), lr=1, max_iter=20)

# L-BFGS requires a different training loop
losses = []
for epoch in tqdm(range(100)):  # Fewer epochs needed
    def closure():
        optimizer.zero_grad()
        pred = model(torch.FloatTensor(X_train))
        loss = loss_fn(pred, torch.FloatTensor(y_train))
        loss.backward()
        losses.append(loss.item())
        return loss
    
    optimizer.step(closure)
    
    if epoch % 10 == 0:
        with torch.no_grad():
            pred = model(torch.FloatTensor(X_train))
            loss = loss_fn(pred, torch.FloatTensor(y_train))
            print(f"Epoch {epoch}: Loss = {loss.item():.8f}")

  0%|          | 0/100 [00:00<?, ?it/s]/Users/lunatang/Projects/Urops/Drake/code/ik-net-optimization/drake-env/lib/python3.13/site-packages/torch/optim/lbfgs.py:457: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  loss = float(closure())
  1%|          | 1/100 [00:00<00:49,  1.98it/s]

Epoch 0: Loss = 1.76009309


 11%|█         | 11/100 [00:04<00:39,  2.26it/s]

Epoch 10: Loss = 1.35387075


 21%|██        | 21/100 [00:09<00:35,  2.24it/s]

...

 81%|████████  | 81/100 [00:35<00:08,  2.32it/s]

Epoch 80: Loss = 0.88928169


 91%|█████████ | 91/100 [00:39<00:04,  2.21it/s]

Epoch 90: Loss = 0.88271171


100%|██████████| 100/100 [00:43<00:00,  2.30it/s]


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
loss_fn = torch.nn.MSELoss()

losses = train_model(model, optimizer, loss_fn, X_train, y_train, epochs=1000, batch_size=2048)
evaluate_model(model, loss_fn, X_test, y_test, dataset, matrix=True)

Backwards Loss: 1.837545
Position Loss: 0.195040
Test Loss: 0.901144


In [ ]:
model

LinearModule(
  (network): Sequential(
    (0): Linear(in_features=7, out_features=40, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=40, out_features=7, bias=True)
  )
)